# STM R

Dans ce deuxieme lab, nous allons reprendre le corpus de l'est republicain mais avec le package STM en R.

Meme dataset que pour le lab 01

Pourquoi le package STM en R?

http://www.structuraltopicmodel.com/

* Facile à utiliser
* Beaucoup de fonctionnalités: preparation du corpus, visualization, ...
* Nombre de topics suggérés et analysable


# Le code R :

1. Chargement du dataset dans une dataframe
2. Nettoyage du code
3. Filtrage des mots en fonction de leur fréquence
4. STM / LDA
5. Résultats
6. Bonheur / Bliss 



In [2]:
# set working directory
setwd("~/UPEM-NUMI/lab02/notebooks/")

# Chargement des packages
source('initialize.R')

# initialisation des parametres de l'experience
source('config.R')

ERROR: Error in library("stm"): there is no package called ‘stm’


In [2]:
# ------------------------------------------------------------------------------
#  Load data
# ------------------------------------------------------------------------------
qqcat("Load data from @{s.input_file}")

# Chargement du corpus dans une dataframe
df <- read.csv(s.input_file, nrows = s.max_rows, col.names = c(s.text_feature))

# combien de rows
qqcat("dimensions:")
dim(df)

# Vérifier le contenu
qqcat(" 2 premiers paragraphes")
df$text[0:2]


[05:52:52] Load data from /Users/alexis/amcp/upem/UPEM-NUMI/lab02/data/estrepublicain_annee_1999.csv
[05:52:56] dimensions:

[1] 10000     1


[05:52:56]  2 premiers paragraphes

[1] André Bauer, le Bonhomme de St-Dié ; Alain Dagosto, Bières de Vézelize (54) ; Paulette Gay, Pieds de cochons de Dommartin ; Corinne Dexemple, Tourte néocastrienne ; Bernadette Paulin, Pâté lorrain de Châtenois ; Andrée Labrux, Pétrou de Senones ; Marie-Thérèse Muller, Cochonneux de la Seille de Sillegny (57) ; Ginette Laporte, Eau de Contrexéville ; Danièle Richard, Framboise saulxuronne ; Josette Pouchucq, Cuisses de grenouilles de Vittel ; Daniel Léonard, Nostre damme de Chiney (Belgique) ; Sophie Valdenaire, Pissenlit de Xertigny ; Simone Leick, Cochon d'autrefois de Sierck les Bains (57) ; Yves Lievens, Kuulkappers de St-Gilles (Belgique) ; Brigitte Lievens, Miel de montagne de Plombières ; Louise Fallot, Madeleine de Commercy ; Claude Himbert, Macaron et bergamote de Nancy ; Robert Fuchs, Marmite d'or ; Jean-Pierre Ruspini, Saumon de Salm ; Jean-Marie Mougel, Andouille du Val d'Ajol ; Jean-François Ancel, Rognons blancs de Raon aux Bois ; Evelyne Bailleux, Fine coquille de Poseïdon de Recey sur Ource (21) ; Dominique Thaller, Tête de veau de Rambervillers ; Nicole Dubois, Boudin noir de Soissons (02) ; Martine Pattey, Image d'Epinal.
[2] Le Smash Entente Club de Lunéville (SECL) a repris ses activités au complexe Berte. Les séances d'entraînement du mercredi soir, de 20 h à 22 h, sont ouvertes à toute personne adulte désirant pratiquer le volley-ball dans un esprit de loisir et de détente. Pour ceux ou celles qui souhaitent participer au championnat loisir, un entraînement spécifique leur est proposé le lundi soir, aux mêmes horaires. L'assemblée générale du club aura lieu le mercredi 29 septembre, à 20 h, dans la salle de réunion de la maison des sports. Tous les membres du club y sont conviés ainsi que toutes les personnes intéressées. Pour tout renseignement complémentaire, s'adresser au président J. Ravaine (03.83.74.12.28).                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
10000 Levels: _ « Je protège toujours mon entraîneur et son staff. Nous sommes tous sur le même bateau et on doit ramer dans le même sens en compagnie des joueurs. Maintenant, vu les moyens engagés et les installations mises à disposition, je ne peux rien donner de plus. Il faut que le doute disparaisse, que le groupe se stabilise, prenne des points et se montre plus conquérant. Quant au recrutement d'un libero, nous n'avons pas trouvé l'élément susceptible de remplacer Jean-Guy Wallemme. Si les techniciens tombent d'accord sur un nom, je ne cracherai pas dessus. Mais j'estime que nous disposons pour la D2 d'un effectif suffisamment bien armé. Maintenant, je le répète, c'est aux joueurs d'aller chercher une victoire impérative face à Caen samedi ». ...

# Pre processing


Le package stm fait tout!


```processed``` contient

* ```processed$documents``` la matrice mots -documents
* ```processed$vocab``` la liste des mots


### Parametres

* lowercase
* removestopwords: utilise le snowball stemmer (meme que NLTK) cf: http://snowball.tartarus.org/algorithms/french/stop.txt
* ajouter des mots avec customstopwords
* stem: stemming transforme le mot en sa racine != lematization



In [4]:
# ------------------------------------------------------------------------------
# 1) pre process the text with basic NLP massaging
# ------------------------------------------------------------------------------
qqcat("pre processing\n")
processed <- textProcessor(df[,s.text_feature],
                           language         = "fr",
                           lowercase        = FALSE,
                           removestopwords  = FALSE,
                           # customstopwords  = c('des'),
                           removenumbers    = FALSE,
                           removepunctuation = FALSE,
                           wordLengths      = c(s.min_wordlen,Inf),
                           striphtml        = FALSE,
                           stem             = FALSE,
                           verbose          = FALSE,
                           metadata         = df)
print(processed)





[05:55:30] pre processing
A text corpus with 10000 documents, and an 123532 word dictionary.


## Reduire le corpus

Exclure les mots trop frequents ou pas assez avec ```lower.thresh``` et ```upper.thresh```



In [5]:
out   <- prepDocuments(processed$documents,
                       processed$vocab,
                       processed$meta,
                       lower.thresh = s.thresh.lower,
                       upper.thresh = s.thresh.upper
                   )

docs  <- out$documents
vocab <- out$vocab
meta  <- out$meta


Removing 118463 of 123532 terms (282553 of 798531 tokens) due to frequency 
Removing 1 Documents with No Words 
Your corpus now has 9999 documents, 5069 terms and 515978 tokens.

# Le topic model

On peut specifier un nombre de topic comme avec Gensim

Mais aussi laisser le modele trouver le nombre de topic optimal en donnant la valeur 0




In [6]:
qqcat("fit stm\n")

num_topics <- 0
fit <- stm(out$documents, out$vocab, 
           num_topics,
           reportevery = 10,
           max.em.its  = 100,
           emtol       = 1.5e-4,
           init.type   = "Spectral",
           seed        = 1
       )

# Sauver l'environnement
qqcat("stm done\n")
if (s.save_envt){
  qqcat("saving to @{s.envt_filename}")
  save.image(s.envt_filename)
}



[05:56:15] fit stm
Beginning Spectral Initialization 
	 Calculating the gram matrix...
	 Finding anchor words...
 	
	 Recovering initialization...
 	..................................................
Initialization complete.
.....................................................................................................
Completed E-Step (20 seconds). 
Completed M-Step. 
Completing Iteration 1 (approx. per word bound = -6.949) 
.....................................................................................................
Completed E-Step (17 seconds). 
Completed M-Step. 
Completing Iteration 2 (approx. per word bound = -6.815, relative change = 1.928e-02) 
.....................................................................................................
Completed E-Step (16 seconds). 
Completed M-Step. 
Completing Iteration 3 (approx. per word bound = -6.769, relative change = 6.718e-03) 
...................................................................................

# Explorer les resultats

* plot(fit, labeltype=c("frex"), main = 'Topic Most Frequent Words',bty="n")



In [ ]:
plot(fit, labeltype=c("frex"), main = 'Topic Most Frequent Words',bty="n")